In [2]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import datetime
import plotly.express as px
import math
import warnings
warnings.filterwarnings('ignore')
import time
from time import perf_counter
import glob
import tweets_utils
import csv
import importlib
import seaborn as sns
%autosave 3600

Autosaving every 3600 seconds


In [3]:
path = glob.glob("C:/Users/gianl/Desktop/Gi/Supsi/BachelorProject/csv/not_found/")
issues = pd.read_csv(path[0] + "users_not_found.csv", low_memory=False, lineterminator="\n")
suspended = pd.read_csv(path[0] + "users_not_found_inspected.csv", low_memory=False, lineterminator="\n")

In [9]:
issues.loc[(issues["code"].notna())]

,id,screen_name,not_found,code,message
649,6044332,CookKCEgyptian,True,63.0,User has been suspended.
791,7154912,Jeapam,True,50.0,User not found.
792,7154912,xiscan1,True,50.0,User not found.
830,7404072,Robert33829889,True,50.0,User not found.
1483,12004852,marshalotis,True,63.0,User has been suspended.
...,...,...,...,...,...
99215,1257676687359254528,TravelWithMOW,True,50.0,User not found.
99240,1258056656728076288,SoloPocono2,True,50.0,User not found.
99288,1258831196479504384,teresawonline,True,63.0,User has been suspended.
99299,1259069865903763456,Mista_Craw4ed,True,50.0,User not found.


In [10]:
ids_suspended = list(suspended.loc[(suspended['code']==63)]["id"])

In [13]:
issues[issues.id.isin(ids_suspended)]

,id,screen_name,not_found,code,message
649,6044332,CookKCEgyptian,True,63.0,User has been suspended.
1483,12004852,marshalotis,True,63.0,User has been suspended.
1921,14194329,KatwalBibika,True,63.0,User has been suspended.
2743,14992551,parabuy,True,63.0,User has been suspended.
3408,15720622,jhakailashkumar,True,63.0,User has been suspended.
...,...,...,...,...,...
99072,1255854351425421313,SatiricusRex,True,63.0,User has been suspended.
99141,1256636223604416512,TPE_connect,True,63.0,User has been suspended.
99195,1257344497463287808,FinTechZoom,True,63.0,User has been suspended.
99288,1258831196479504384,teresawonline,True,63.0,User has been suspended.
